In [ ]:
#!pip install matplotlib



In [ ]:
import time
from dataclasses import dataclass

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List,Literal
from helper_functions import *

import cudaq

In [ ]:
#test kernel
depth = 2
n_qubits = 5
seed = 123
rng = np.random.default_rng(seed)
thetas = rng.uniform(0, 2*np.pi, size=(depth*n_qubits)).astype(float)

@cudaq.kernel
def circuit(n_qubits: int, depth: int, thetas: List[float]):
    q = cudaq.qvector(n_qubits)
    # rotate
    for d in range(depth):
        for i in range(n_qubits):
            ry(thetas[d*n_qubits +i], q[i])

        # entangle
        for i in range(n_qubits-1):
            x.ctrl(q[0], q[i+1])


print(cudaq.draw(circuit,n_qubits, depth,thetas))


In [ ]:
@dataclass
class Result:
    target: str
    n_qubits: int
    depth: int
    min_time: float
    avg_time: float
    std_time: float
    ok: bool
    error: str = ""

def time_statevector(target: str, n_qubits: int, depth: int, seed: int = 0, reps: int = 5) -> Result:
    rng = np.random.default_rng(seed)
    thetas = rng.uniform(0, 2*np.pi, size=(depth*n_qubits)).astype(float)
    cudaq.set_target(target)
    try:
        cudaq.set_target(target)
        circuit(n_qubits, depth, thetas)

        times = []
        for _ in range(reps):
            t0 = time.perf_counter()
            _ = cudaq.get_state(circuit,n_qubits, depth, thetas)
            t1 = time.perf_counter()
            times.append(t1 - t0)

        return Result(target, n_qubits, depth, float(min(times)), float(np.mean(times)), float(np.std(times)), True)

    except Exception as e:
        return Result(target, n_qubits, depth, float("nan"), float("nan"),float("nan"), False, str(e))
    
def check_crossover(sweep_param : Literal["n", "d"], 
                    sweep_range : List[int],
                    const_param_val :int = 30, 
                    seed :int = 123, 
                    reps :int = 5,
                    plot :bool = True,
                    plot_std: bool = False,
                    show_df: bool = False) -> pd.DataFrame:
    rows = []
    for val in sweep_range:
        if sweep_param=="n":
            cpu = time_statevector(target="qpp-cpu", 
                                   n_qubits=val, 
                                   depth=const_param_val, 
                                   seed=seed, 
                                   reps=reps)
            gpu = time_statevector(target="nvidia", 
                                   n_qubits=val, 
                                   depth=const_param_val, 
                                   seed=seed, 
                                   reps=reps)
            print(f"{sweep_param}={val:2d} | CPU {cpu.min_time:8.4f}s {'OK' if cpu.ok else 'FAIL'} | GPU {gpu.min_time:8.4f}s {'OK' if gpu.ok else 'FAIL'}")

        elif sweep_param =="d":
            cpu = time_statevector(target="qpp-cpu", 
                                   n_qubits=const_param_val, 
                                   depth=val, 
                                   seed=seed, 
                                   reps=reps)
            gpu = time_statevector(target="nvidia", 
                                    n_qubits=const_param_val, 
                                    depth=val, 
                                    seed=seed, 
                                    reps=reps)
            print(f"{sweep_param}={val:2d} | CPU {cpu.min_time:8.4f}s {'OK' if cpu.ok else 'FAIL'} | GPU {gpu.min_time:8.4f}s {'OK' if gpu.ok else 'FAIL'}")

        rows += [cpu.__dict__, gpu.__dict__]
    df = pd.DataFrame(rows)
    if plot:
        plot_crossoverv2(df, sweep_param = sweep_param, plot_std=plot_std)
    if show_df:
        print(df)
    return pd.DataFrame(rows)



In [ ]:
res = check_crossover(sweep_param="n",
                      sweep_range=list(range(1, 13)), 
                      const_param_val=2, 
                      seed=123, 
                      reps=5, 
                      plot=True,
                      plot_std=False,
                      show_df=True)

In [ ]:
res = check_crossover(sweep_param="d",
                      sweep_range=list(range(1, 15)), 
                      const_param_val=6, 
                      seed=123, 
                      reps=10, 
                      plot=True,
                      show_df=True)